# PROJECT-1 : Multiclass Multilabel prediction For stack overflow Questions

In [7]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [8]:
!wget https://www.dropbox.com/s/5721wcs2guuykzl/stacksample.zip?dl=0

--2025-12-28 20:33:25--  https://www.dropbox.com/s/5721wcs2guuykzl/stacksample.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/bnabda9cee2yuyi6wesmv/stacksample.zip?rlkey=sucdc1cm2m7fnwtzhykv24g8z&dl=0 [following]
--2025-12-28 20:33:26--  https://www.dropbox.com/scl/fi/bnabda9cee2yuyi6wesmv/stacksample.zip?rlkey=sucdc1cm2m7fnwtzhykv24g8z&dl=0
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca7e1759748eb50f1f33f9bed53.dl.dropboxusercontent.com/cd/0/inline/C3743slkyJAfODpuKKer53hiOX3d3pd5SBkT7WqO6idG3H5BzCH9nWkbEE_sQvGjzb_92zlxQDdgJRFklHqY17AdhAegcsCGnI98LoLZPnrYsvY4f9ItLjZEbsb9d2Lf1Qg/file# [following]
--2025-12-28 20:33:26--  https://uca7e1759748eb50f1f33f9bed53.dl.dropboxusercontent.com/cd/0/in

In [9]:
!unzip stacksample.zip?dl=0

Archive:  stacksample.zip?dl=0
  inflating: Answers.csv             
  inflating: Questions.csv           
  inflating: Tags.csv                


In [10]:
import warnings
warnings.filterwarnings('ignore')

In [11]:
import pandas as pd
import numpy as np
import re

In [12]:
Questions = pd.read_csv('/content/Questions.csv',encoding='latin-1')
Tags = pd.read_csv('/content/Tags.csv',encoding='latin-1')

In [13]:
Questions.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [14]:
Tags.head()

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn


In [15]:
data=Questions.merge(Tags, on='Id')

In [16]:
data=data[['Id','Body','Tag']]

In [17]:
data.head()

,Id,Body,Tag
0,80,<p>I've written a database generation script i...,flex
1,80,<p>I've written a database generation script i...,actionscript-3
2,80,<p>I've written a database generation script i...,air
3,90,<p>Are there any really good tutorials explain...,svn
4,90,<p>Are there any really good tutorials explain...,tortoisesvn


In [18]:
print(data['Body'])

0          <p>I've written a database generation script i...
1          <p>I've written a database generation script i...
2          <p>I've written a database generation script i...
3          <p>Are there any really good tutorials explain...
4          <p>Are there any really good tutorials explain...
                                 ...                        
3750989    <p>I have input which I use to filter my array...
3750990    <p>I have input which I use to filter my array...
3750991    <p>I'm running a mocha test and I noticed my c...
3750992    <p>I'm running a mocha test and I noticed my c...
3750993    <p>I'm running a mocha test and I noticed my c...
Name: Body, Length: 3750994, dtype: object


In [19]:
def clear_text(text):
  text=text.lower()
  text=re.sub(r'<code> .*?</code>','',text,flags=re.S)
  text=re.sub(r'<.*?>','',text)
  text=re.sub(r'<http\S+','',text)
  text=re.sub(r'[^a-z ]','',text)
  return text

In [20]:
data['Body']=data['Body'].astype(str).apply(clear_text)

In [21]:
print(data['Body'])

0          ive written a database generation script in sq...
1          ive written a database generation script in sq...
2          ive written a database generation script in sq...
3          are there any really good tutorials explaining...
4          are there any really good tutorials explaining...
                                 ...                        
3750989    i have input which i use to filter my array of...
3750990    i have input which i use to filter my array of...
3750991    im running a mocha test and i noticed my chang...
3750992    im running a mocha test and i noticed my chang...
3750993    im running a mocha test and i noticed my chang...
Name: Body, Length: 3750994, dtype: object


In [22]:
top_tags=data['Tag'].value_counts().head(10).index.tolist()
data=data[data['Tag'].isin(top_tags)]

grouped=data.groupby('Id').agg({
    'Body':'first',
    'Tag': lambda x: list(set(x))
}).reset_index()


In [23]:
from sklearn.preprocessing import MultiLabelBinarizer

In [24]:
mlb=MultiLabelBinarizer(classes=top_tags)

In [25]:
X=grouped['Body']
Y=mlb.fit_transform(grouped['Tag'])

In [26]:
print(X)
print('*'*50)
print(Y)

0         i have a little game written in c it uses a da...
1         i am working on a collection of classes used f...
2         i would like the version property of my applic...
3         whats the simplest way to connect and query a ...
4         i need to grab the baseencoded representation ...
                                ...                        
706331    im trying to detect the flash out of date erro...
706332    i need to extend a shell script bash as i am m...
706333    i am building a custom mvc project and i have ...
706334    under minifyenabled i changed from false to tr...
706335    i have input which i use to filter my array of...
Name: Body, Length: 706336, dtype: object
**************************************************
[[0 0 1 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]


In [27]:
from sklearn.model_selection import train_test_split

In [28]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

In [29]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping


In [30]:
tokenizer = Tokenizer(num_words=30000, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)

X_train_pad = pad_sequences(X_train_seq, maxlen=200)
X_val_pad = pad_sequences(X_val_seq, maxlen=200)



In [34]:
model = Sequential([
    Embedding(30000, 128 ,input_length=200),
    Bidirectional(LSTM(64)),
    Dense(64, activation='relu'),
    Dense(10, activation='sigmoid')
])

In [35]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [36]:
es=EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

model.fit(
    X_train_pad, Y_train,
    validation_data=(X_val_pad, Y_val),
    epochs=5,
    batch_size=64,
    callbacks=[es]
)

Epoch 1/5
8830/8830 ━━━━━━━━━━━━━━━━━━━━ 229s 25ms/step - accuracy: 0.5122 - loss: 0.2317 - val_accuracy: 0.7750 - val_loss: 0.1119
Epoch 2/5
8830/8830 ━━━━━━━━━━━━━━━━━━━━ 219s 25ms/step - accuracy: 0.7933 - loss: 0.1031 - val_accuracy: 0.7827 - val_loss: 0.1055
Epoch 3/5
8830/8830 ━━━━━━━━━━━━━━━━━━━━ 221s 25ms/step - accuracy: 0.8128 - loss: 0.0910 - val_accuracy: 0.7886 - val_loss: 0.1043
Epoch 4/5
8830/8830 ━━━━━━━━━━━━━━━━━━━━ 223s 25ms/step - accuracy: 0.8287 - loss: 0.0815 - val_accuracy: 0.7859 - val_loss: 0.1076
Epoch 5/5
8830/8830 ━━━━━━━━━━━━━━━━━━━━ 220s 25ms/step - accuracy: 0.8432 - loss: 0.0721 - val_accuracy: 0.7837 - val_loss: 0.1146


In [37]:
y_pred_prob=model.predict(X_val_pad)
y_pred=(y_pred_prob>0.5).astype(int)

4415/4415 ━━━━━━━━━━━━━━━━━━━━ 44s 10ms/step


In [39]:
from sklearn.metrics import hamming_loss, f1_score, precision_score, recall_score

In [41]:
print('Hamming Loss :', hamming_loss(Y_val, y_pred))
print('Micro F1 :', f1_score(Y_val, y_pred, average='micro'))
print('Macro F1 :', f1_score(Y_val, y_pred, average='macro'))
print('Micro Precision :', precision_score(Y_val, y_pred, average='micro'))
print('Micro Recall :', recall_score(Y_val, y_pred, average='micro'))

Hamming Loss : 0.04039556021179602
Micro F1 : 0.8160099046292535
Macro F1 : 0.8133350673832862
Micro Precision : 0.8748971591734017
Micro Recall : 0.764549864968553


In [54]:
def predict_tags(text):
  text=clear_text(text)
  seq=tokenizer.texts_to_sequences([text])
  pad=pad_sequences(seq, maxlen=200)
  pred= (model.predict(pad)>0.5).astype(int)
  return mlb.inverse_transform(pred)[0]

In [55]:
predict_tags('i wrote this code in python using keras with javascript')
predict_tags('i wrote this code in java, c#, php, android in SQL')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


('java',)

In [56]:
top_tags

['javascript',
 'java',
 'c#',
 'php',
 'android',
 'jquery',
 'python',
 'html',
 'c++',
 'ios']